Note: This is only a **temporary** notebook that is used for development purposes. 

In [42]:
%load_ext autoreload
%autoreload 2


import os
import sys
import numpy as np
import pandas as pd

from config import config
import fit_ac_fc as f

In [200]:
ab = f.load_abundance_thresholds(os.path.join(config.A549_test, \
        "A549_abundance_thresholds.txt"))
tps = f.load_timepoint_counts(os.path.join(config.A549_test, \
        "A549_timepoint_counts.txt"))

In [201]:
_tps, names = f.convert_timepoint_counts(tps)
_abundance = f.convert_abundance_thresholds(ab)

In [202]:
_tps.head()

reps,1,2,1,2,1,2,1,2
time,3,3,14,14,21,21,28,28
0,133.0,168.0,1.0,46.0,0.0,48.0,0.0,1.0
1,789.0,1046.0,786.0,779.0,1181.0,696.0,724.0,1179.0
2,515.0,709.0,404.0,442.0,193.0,424.0,205.0,442.0
3,520.0,549.0,326.0,1265.0,244.0,1341.0,135.0,3171.0
4,584.0,433.0,255.0,193.0,180.0,527.0,188.0,685.0


In [203]:
_abundance

reps,1,2,1,2,1,2,1,2
time,3,3,14,14,21,21,28,28
log2CountsThresh,3.335947,3.475,3.33828,4.089135,4.525,4.325,4.225,3.575


In [223]:
_tps.shape

(24908, 8)

In [237]:
mask = pd.DataFrame(_tps.values > _abundance.values, columns=_tps.columns)

In [247]:
data = []
for i in mask.columns.levels[0]: 
    data.append((mask[i].sum(axis=1) > 2).values)

In [252]:
pd.DataFrame(np.vstack(data), index=mask.columns.levels[0]).T

reps,1,2
0,False,True
1,True,True
2,True,True
3,True,True
4,True,True
5,True,True
6,True,True
7,True,True
8,True,True
9,True,True


# fit_ac_fc function

In [263]:
def fit_ac_fc(counts, abundance, n_good=2): 
    assert (counts.columns == abundance.columns).all()
    
    
    pass_mask = pd.DataFrame(counts.values > abundance.values, columns=counts.columns)
    
    data = []
    for i in pass_mask.columns.levels[0]: 
        data.append((mask[i].sum(axis=1) >= n_good).values)
        
    pass_mask = pd.DataFrame(np.vstack(data), index=mask.columns.levels[0]).T
    
    return pass_mask

In [265]:
%time fit_ac_fc(_tps, _abundance)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 12.6 ms


reps,1,2
0,False,True
1,True,True
2,True,True
3,True,True
4,True,True
5,True,True
6,True,True
7,True,True
8,True,True
9,True,True
